In [ ]:
! pip install python-telegram-bot==13.1

In [ ]:

! pip install openai==0.28

### ChatGPT

In [ ]:
import openai

openai.api_key = 'SECRET_TOKEN'

def get_code(input_message, file_name, column_names, column_types, save_directory, save_name):
    """
    input_message - промпт с заданием для chatGPT (string)

    file_name - имя датафрейма, для которого выполняется промпт (string)

    column_names - список с именами столбцов в датафрейме

    column_types - словарь с типами столбцов вида {'имя столбца': 'формат данных столбца'} - ТУТ МОЖЕШЬ СДЕЛАТЬ КАК ТЕБЕ УДОБНЕЕ

    save_directory - директория для сохранения результатов

    save_name - с каким именем надо сохранить результат
    """

    messages = [{'role': 'assistant', 'content': f"""Forget all previous messages from me.
                                                    You are a senior python developer, your job is to carefully
                                                    follow the given instructions to solve the require.""" + input_message +
                                                    f""". The pd.DataFrame() object is placed in the {file_name} variable.
                                                    This dataframe has columns {column_names}. Columns are defined as those
                                                    data types: {column_types}. If the result of the program includes a picture (png files),
                                                    write a code for saving graphs locally as png files, if the result is a text or table,
                                                    write a code for saving it locally as excel file. Save those files
                                                    in the directory {save_directory} with name {save_name}""" +
                                                    """ Make sure to apply those restriction and follow them directly and
                                                    do not skip any of them: 1. output with no introduction, no explaintation, only code;
                                                    2. DONT MAKE ANY MISTAKES, check if you did any;
                                                    3. only return python3 code, and nothing else;
                                                    4. NO DOCUMENATION IN THE OUTPUT"""}]

    completion = openai.ChatCompletion.create(model='gpt-3.5-turbo',
                                        messages=messages,
                                        temperature=0.5,
                                        max_tokens=1000)

    generated_code = completion['choices'][0]['message']['content']

    if generated_code.find('```python\n') != -1:

        output_code = generated_code.split('```python\n')[1].split('```')[0]

    else:

        output_code = generated_code

    print(output_code)

    return output_code

### Функция предобработки данных

Определение типа файла и чтение

In [ ]:
def load(names, usecols = None, log = False, append = False, mem_reduce = False, tqdm_info = False):
  def load_data(name):
    name = name.replace('//', '\\')
    d = None
    if log:
      print(name + 'is loading!')
    if ('.xlsx' in name) or ('.xls' in name):
      d = pd.read_excel(name)
    elif '.csv' in name:
      try:
        d = csv.read_csv(name, read_options = pa.csv.ReadOptions(use_threads = True, column_names = usecols),
                         parse_options = pa.csv.ParseOptions(delimiter = ';', ignore_empty_lines = True))
        if len(d.column_names) == 1:
          d = csv.read_csv(name, read_options = pa.csv.ReadOptions(use_threads = True, column_names = usecols),
                         parse_options = pa.csv.ParseOptions(delimiter = ',', ignore_empty_lines = True))
          if len(d.column_names) == 1:
            d = csv.read_csv(name, read_options = pa.csv.ReadOptions(use_threads = True, column_names = usecols),
                         parse_options = pa.csv.ParseOptions(delimiter = ';', ignore_empty_lines = True))

        d = d.to_pandas(use_threads = True)

      except:
        try:
          d = pd.read_csv(name, sep=';', usecols = usecols, encoding = 'utf-8_sig')
          if len(d.columns) == 1:
            d = pd.read_csv(name, sep=',', usecols = usecols, encoding = 'utf-8_sig')
            if len(d.columns) == 1:
              d = pd.read_csv(name, sep='\t', usecols = usecols, encoding = 'utf-8_sig')

        except:
          try:
            d = pd.read_csv(name, sep=';', usecols = usecols, encoding = 'cp1251')
            if len(d.columns) == 1:
              d = pd.read_csv(name, sep=',', usecols = usecols, encoding = 'cp1251')
              if len(d.columns) == 1:
                d = pd.read_csv(name, sep='\t', usecols = usecols, encoding = 'cp1251')
          except:
            print('ERROR!!! ', name)
    elif '.frt' in name or '.feather' in name:
      try:
        d = pd.read_feather(name, use_threads = True, columns = usecols)
      except ArrowInvalid:
        d = pd.DataFrame((feather.read_table(name, use_threads = True)).to_pylist())
    elif '.pkl' in name:
      with open(name, 'rb') as fp:
        try:
          d = pickle.load(fp)
        except:
          print(f'Fail to load {name}')
          d = pd.DataFrame()
    elif '.parquet' in name:
      d = pd.read_parquet(name)
    elif '.json.gz' in name:
      d = pd.read_json(name, lines = True, compression = 'gzip')
    elif '.json' in name:
      with open(name, 'r') as fp:
        d = json.load(fp)
    if log:
      print(name + ' is loaded!')
    return d
  if type(names) == str:
    d = load_data(names)
  else:
    return 0
  if type(d) == list and len(d) == 1:
    return d[0]
  else:
    return d


Приведение к правильному типу столбцов датафрейма

In [ ]:
def clean_dataframe(df):
    all_cols = df.columns
    object_cols = df.select_dtypes(include=['object']).columns.tolist()

    for col in all_cols:
      if col in object_cols:
        try:
          df[col] = (
                df[col].astype(str)
                .str.replace(",", ".")
                .replace(["-", "", " ", "nan", "<NA>", "None"], np.nan)
            )
          df[col] = df[col].astype('float64')
        except:
          df[col] = df[col]
          pass
      else:
        df[col] = df[col]

    return df

Удаление выбросов в датафрейме

In [ ]:
def remove_outliers(df):
    numeric_cols = df.select_dtypes(include=['number']).columns

    for col in numeric_cols:
        q1 = df[col].quantile(0.25)
        q3 = df[col].quantile(0.75)
        iqr = q3 - q1
        lower_bound = q1 - 1.5 * iqr
        upper_bound = q3 + 1.5 * iqr
        df = df[(df[col] >= lower_bound) & (df[col] <= upper_bound)]

    return df

### Telegram Bot

In [ ]:
import os
import logging

from sqlalchemy import create_engine

from telegram.ext import CommandHandler, Updater, Filters, MessageHandler

import pandas as pd
import tqdm
import pyarrow as pa
import pyarrow.feather as feather
import xml.etree.ElementTree as ET
import json
import pickle

from tqdm import tqdm
from datetime import timedelta
from pyarrow import csv
from pyarrow.lib import ArrowInvalid
import hashlib

import numpy as np
import re
import datetime
import time
import concurrent.futures

In [ ]:
TOKEN = 'TG_TOKEN'

'''Конфигурация бота'''
logging.basicConfig(
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
    level=logging.INFO)


def wake_up(update, context):
    chat = update.effective_chat
    name = update.message.chat.first_name

    context.bot.send_message(
        chat_id=chat.id,
        text='Привет, {}. Загрузи файл с информацией в формате (название, URL, xpath запрос)'.format(name),
    )


def handle_message(update, context):

    now = datetime.datetime.now()

    chat = update.effective_chat

    local_dir = '/content/'

    input_message = update.message.caption

    print(input_message)

    if update.message.document is not None:

        doc_message = update.message.document

        file_name = doc_message["file_name"]

        context.bot.get_file(doc_message).download(file_name)

    data = load(local_dir + file_name)
    data = clean_dataframe(data)
    data = remove_outliers(data)
    column_types = dict()
    column_names = data.columns.tolist()

    for name in column_names:
      column_types[name] = type(data[name][0])

    data.to_csv(f'/content/correct_{file_name}')

    count = 0

    check = False

    while check is False and count < 5:

      output_code = get_code(input_message,
                          f'/content/correct_{file_name}',
                          column_names,
                          column_types,
                          '/content/sample_data/results',
                          'result_test')

      try:
        print(count, '\n')

        exec(output_code)

        break

      except:

        count += 1

    folder = '/content/sample_data/results'

    if len(os.listdir(folder)) != 0:
      for index, filename in enumerate(os.listdir(folder)):
          file_path = os.path.join(folder, filename)
          with open(file_path, 'rb') as f:
              update.message.reply_document(f, caption=f'Файл {index+1}')
          try:
              if os.path.isfile(file_path) or os.path.islink(file_path):
                  os.unlink(file_path)
          except Exception as e:
              print(f'Не удалось удалить {file_path}. Причина: {e}')

    else:
      context.bot.send_message(
        chat_id=chat.id,
        text='Переформулируйте задачу',
    )



def main():
    updater = Updater(token=TOKEN, use_context=True)

    updater.dispatcher.add_handler(CommandHandler('start', wake_up))

    updater.dispatcher.add_handler(
         MessageHandler(Filters.document, handle_message))

    updater.start_polling(poll_interval=0.10)
    updater.idle()


if __name__ == '__main__':
    main()